# Netflix Movie Recommendation

## Overview data

In [2]:
import pandas as pd
import numpy as np
import warnings
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from collections import Counter
warnings.filterwarnings("ignore")

In [9]:
data = pd.read_csv('netflix_titles.csv')

# Data Analysis and Data cleaning

In [10]:
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [12]:
# To be used to compare similar movies
description = data[['description','listed_in']]

In [760]:
#The given number of features and data points
#no. of data points = 7787
#no. of features = 12
data.shape

(7787, 12)

In [761]:
#finding out all feature names
data.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [762]:
#Keeping Relevant features to used for recommendation system building using nlp
data = data[['title','country','director','cast','listed_in','description']]

#### Missing data

In [763]:
#removing all Nan title null(if movie name not present means movie does not exist in list) 
data = data[~data['title'].isnull()] 
data.shape

(7787, 6)

In [764]:
#description required for words based vector formation for finding similarity between two movies
data[~data['description'].isnull()]
data.shape

(7787, 6)

#### Duplicate data

In [765]:
#checking for title duplicates = none exists as the sum is 0
#Therefore there are no duplicate title in movie/TV series data to remove
sum(data.duplicated('title'))

0

In [766]:
#does not make sense to business sense to remove duplicate cast,listed_in(basically genre) 
#as they might be the same for two different movies

In [767]:
# Trying filling for all NAN values with an empty string
data.fillna(" ", inplace = True)
data.shape

(7787, 6)

In [768]:
#Saving data we are going to work with
data.to_pickle('Pickle/7k_data_points')

In [944]:
data = pd.read_pickle('Pickle/7k_data_points')

#### Basic stats for data

In [945]:
#There area a total of 6832 unique actor casts in our movies
#718 movies whos cast are not known in our data
data['cast'].describe()

count     7787
unique    6832
top           
freq       718
Name: cast, dtype: object

In [946]:
#There area a total of 4050 unique directors
#2389 movies whos directors are not known in our data
data['director'].describe()

count     7787
unique    4050
top           
freq      2389
Name: director, dtype: object

In [947]:
director_count = Counter(data['director'])
director_count.most_common(10)

[(' ', 2389),
 ('Raúl Campos, Jan Suter', 18),
 ('Marcus Raboy', 16),
 ('Jay Karas', 14),
 ('Cathy Garcia-Molina', 13),
 ('Youssef Chahine', 12),
 ('Martin Scorsese', 12),
 ('Jay Chapman', 12),
 ('Steven Spielberg', 10),
 ('David Dhawan', 9)]

# Text Preprocessing

#### Taking care of Stopwords

In [948]:
stop_words = stopwords.words('english')
print('These are all the stop words', list(stop_words))
def nlp_processing(text,index,column):
    string = " "
    for word in text.split():
        words = ("".join(e for e in word if e.isalnum()))
        words.lower()
        if words not in stop_words:
            string += ' ' + words 
    data[column][index] = string

These are all the stop words ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', '

In [949]:
data['description'] = data['description'].map(lambda x : x.lower())

In [950]:
for index,row in data.iterrows():
    nlp_processing(row['description'],index,'description')

In [951]:
data.head()

,title,country,director,cast,listed_in,description
0,3%,Brazil,,"João Miguel, Bianca Comparato, Michel Gomes, R...","International TV Shows, TV Dramas, TV Sci-Fi &...",future elite inhabit island paradise far cro...
1,7:19,Mexico,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...","Dramas, International Movies",devastating earthquake hits mexico city trap...
2,23:59,Singapore,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...","Horror Movies, International Movies",army recruit found dead fellow soldiers forc...
3,9,United States,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...","Action & Adventure, Independent Movies, Sci-Fi...",postapocalyptic world ragdoll robots hide fe...
4,21,United States,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",Dramas,brilliant group students become cardcounting...


In [952]:
# keeping just the top four cast names
data['cast'] = data['cast'].map(lambda x : x.split(',')[:3])

#seprating the listed_in category
data['listed_in'] = data['listed_in'].map(lambda x : x.lower().split(','))
 
#joining the director surnames and first names
data['director'] = data['director'].map(lambda x: x.replace(' ','').lower().split(' '))

#country
data['country'] = data['country'].map(lambda x : x.replace(' ','').lower().split(' '))

#making a list of description
data['description']= data['description'].map(lambda x : x.split(' '))

In [953]:
#joining the actor surnames and names 
for index,rows in data.iterrows():
    
    rows['cast'] = [x.lower().replace(' ','') for x in rows['cast']] 

In [954]:
data.head()

,title,country,director,cast,listed_in,description
0,3%,[brazil],[],"[joãomiguel, biancacomparato, michelgomes]","[international tv shows, tv dramas, tv sci-f...","[, , future, elite, inhabit, island, paradise,..."
1,7:19,[mexico],[jorgemichelgrau],"[demiánbichir, héctorbonilla, oscarserrano]","[dramas, international movies]","[, , devastating, earthquake, hits, mexico, ci..."
2,23:59,[singapore],[gilbertchan],"[teddchan, stellachung, henleyhii]","[horror movies, international movies]","[, , army, recruit, found, dead, fellow, soldi..."
3,9,[unitedstates],[shaneacker],"[elijahwood, johnc.reilly, jenniferconnelly]","[action & adventure, independent movies, sci...","[, , postapocalyptic, world, ragdoll, robots, ..."
4,21,[unitedstates],[robertluketic],"[jimsturgess, kevinspacey, katebosworth]",[dramas],"[, , brilliant, group, students, become, cardc..."


# Word corpus

### Making a new column containg all the features in a single string  

In [955]:
#Form a column text such that it contains all the columns merged in string format
data['text'] = ''
column = list(data.columns)
column.remove('title')
for index,rows in data.iterrows():
    words = ''
    for col in column:
            words = words + ' '.join(rows[col]) + ' '
    data['text'][index] = words

In [956]:
data['text'][1]

'mexico jorgemichelgrau demiánbichir héctorbonilla oscarserrano dramas  international movies   devastating earthquake hits mexico city trapped survivors walks life wait rescued trying desperately stay alive  '

In [957]:
#replacing the double spaces with single spaces caused because of intitial space in description
data['text'] = data['text'].map(lambda x : x.replace('  ', ' '))

In [958]:
data['text'][1]

'mexico jorgemichelgrau demiánbichir héctorbonilla oscarserrano dramas international movies  devastating earthquake hits mexico city trapped survivors walks life wait rescued trying desperately stay alive '

In [959]:
lst = ['country','director','cast','listed_in','description']
data = data.drop((lst),axis=1) 
data.head()

,title,text
0,3%,brazil joãomiguel biancacomparato michelgomes ...
1,7:19,mexico jorgemichelgrau demiánbichir héctorboni...
2,23:59,singapore gilbertchan teddchan stellachung hen...
3,9,unitedstates shaneacker elijahwood johnc.reill...
4,21,unitedstates robertluketic jimsturgess kevinsp...


In [960]:
data.to_pickle('Pickle/ 7k_Nlp_preprocessed_data')

In [13]:
data = pd.read_pickle('Pickle/ 7k_Nlp_preprocessed_data')

## Bag of words based movie similarity

In [962]:
title_vectorizer = CountVectorizer()
title_features = title_vectorizer.fit_transform(data['text'])
#CountVectorizer().fit_transform(data['title']) returns a sparse matrix with dimensions  =  no. points * words_in_corpus

title_features.shape

(7787, 36439)

In [963]:
def Bag_of_words_model(doc_id,cut_off):
    #doc_id is the number on the new index formed after CountVectorizer is applied to the data['text']
    #pairwise distance will save the distance between given input apparel and all other apparels
    pairwise_dist = pairwise_distances(title_features,title_features[doc_id])
    
    #Gives us indices of data points sorted in ascending order based on the eucledian distances sorted 
    indices = np.argsort(pairwise_dist.flatten())[0:cut_off]
    
    data_indices = list(data.index[indices])
    
    for i in range(0,len(data_indices)):
        print('Title = {}'.format(data['title'].loc[data_indices[i]]))
        print('Story = {}'.format(description['description'].loc[data_indices[i]]))
        print('Listed under = {}'.format(description['listed_in'].loc[data_indices[i]]))
        print('   ')
        

In [964]:
Bag_of_words_model(6487,4)

Title = The Incredibles 2
Story = When Elastigirl gets recruited for a high-profile crime-fighting mission, Mr. Incredible takes on his toughest assignment ever: full-time parenting.
Listed under = Children & Family Movies, Comedies
   
Title = Incredibles 2 (Spanish Version)
Story = When Elastigirl gets recruited for a high-profile crime-fighting mission, Mr. Incredible takes on his toughest assignment ever: full-time parenting.
Listed under = Children & Family Movies, Comedies
   
Title = LEGO Marvel Super Heroes: Guardians of the Galaxy
Story = The Guardians are on a mission to deliver the Build Stone to the Avengers before the Ravagers, Thanos and his underlings steal it from them.
Listed under = Children & Family Movies
   
Title = Tellur Aliens
Story = Three teenaged best friends on the planet Telluria begin an incredible journey of adventure as they fight against the most evil being in the universe.
Listed under = Children & Family Movies
   


## TFIDF based movie similarity model

In [5]:
Tfidf_vectorizer = TfidfVectorizer()
Tfidf_title_features = Tfidf_vectorizer.fit_transform(data['text'])

#Tfidf_vectorizer().fit_transform(data['title']) returns a sparse matrix with dimensions  =  no. points * words_in_corpus
# tfidf_title_features[doc_id, index_of_word_in_corpus] = tfidf values of the word in given doc

In [14]:
def Tfidf_model(doc_id,cut_off):
    #doc_id is the number on the new index formed after TfidfVectorizer is applied to the data['text']
    #pairwise distance will save the distance between given input apparel[doc_id] and all other apparels
    pairwise_dist = pairwise_distances(Tfidf_title_features,Tfidf_title_features[doc_id])
    
    #Gives us indices of data points sorted in ascending order based on the eucledian distances
    indices = np.argsort(pairwise_dist.flatten())[:cut_off]
    
    #data indices in our original data set
    data_indices = list(data.index[indices])
    
    for i in range(0,len(data_indices)):
        print('Title = {}'.format(data['title'].loc[data_indices[i]]))
        print('Story = {}'.format(description['description'].loc[data_indices[i]]))
        print('Listed under = {}'.format(description['listed_in'].loc[data_indices[i]]))
        print('   ')
        

In [15]:
Tfidf_model(6487,4)

Title = The Incredibles 2
Story = When Elastigirl gets recruited for a high-profile crime-fighting mission, Mr. Incredible takes on his toughest assignment ever: full-time parenting.
Listed under = Children & Family Movies, Comedies
   
Title = Incredibles 2 (Spanish Version)
Story = When Elastigirl gets recruited for a high-profile crime-fighting mission, Mr. Incredible takes on his toughest assignment ever: full-time parenting.
Listed under = Children & Family Movies, Comedies
   
Title = Unicorn Store
Story = After failing out of art school and taking a humdrum office job, a whimsical painter gets a chance to fulfill her lifelong dream of adopting a unicorn.
Listed under = Comedies, Dramas, Independent Movies
   
Title = Barely Lethal
Story = A girl raised to be a government assassin just wants to be a normal teen – so she runs away and pretends to be a high school exchange student.
Listed under = Action & Adventure, Comedies
   
